In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import yaml
import cftime
import xarray as xr

#My functions
sys.path.insert(0,'../functions/')
import functions_HeatWavesCities as fun_HWC


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_EUR-11.txt', 'r') as file:  path_eur11 = file.read()
    
dir_CORDEX  = path_eur11
dir_scripts = f'{path_main}Scripts/'
dir_names   = f'{path_main}Scripts/Model_lists/'
dir_JJA     = f'{path_main}Data/EURO-CORDEX/JJA/'
dir_Yearmax = f'{path_main}Data/EURO-CORDEX/Yearmax/'
if not os.path.exists(dir_JJA):      os.mkdir(dir_JJA)
if not os.path.exists(dir_Yearmax):  os.mkdir(dir_Yearmax)


## Prepare variables and parameters

In [ ]:
#Define scenarios and variables
variables = ['tasmax', 'tasmin']


#Define models and RCPs which should be used
all_models = dict()
all_models['rcp85'] = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(eval(line[:-1]))

#Define time periods
time_periods = [[1981, 2010], [2070, 2099]]


## Calculation

In [ ]:
#Loop over time periods
for time_per in time_periods:
    
    #Create slice for selecting data
    time_sel = slice(str(time_per[0]), str(time_per[1]))
    
    #Define scenarios
    if int(time_sel.start)>2005:
        scenarios = ['rcp85']
    else:
        scenarios = ['historical', 'rcp85']

    #Loop over models
    for model in all_models['rcp85']:
        
        print("_".join(model))

        #Loop over variables
        for variab in variables:

            print(" -" + variab, end=': ')

            #Get file names
            fnames_merge = []
            for scen in scenarios:

                dir_files = dir_CORDEX + scen + '/' + variab + '/'
                fnames = [dir_files + file for file in os.listdir(dir_files) if model[0] in file and model[1] in file and model[2] in file]
                fnames = sorted(fnames)

                #Only include files within selected period
                fnames = [fname for fname in fnames if int(fname[-11:-7])>=time_per[0]]
                fnames = [fname for fname in fnames if int(fname[-20:-16])<=time_per[1]]
            
                #Add to list for all files for merging
                fnames_merge = fnames_merge + fnames

            print(fnames_merge[0][-20:-16] + '-' + fnames_merge[-1][-11:-7])

            #Merge single files to one large file
            t_sta = t_util.time()
            file_merge = dir_JJA + 'CORDEX_merged_' + variab + '_' + scen + "_" + "_".join(model) + '_' + "_".join([str(time) for time in time_per]) + '_JJAmean_tmp.nc'
            if os.path.exists(file_merge): os.remove(file_merge)
            os.system('cdo mergetime ' + " ".join(fnames_merge) + " " + file_merge)
            t_end = t_util.time()
            print(" -- " + "{:.1f}".format(t_end - t_sta))

            #Open data set 
            data = xr.open_dataset(file_merge, use_cftime=True)
            
            #Select time
            data = data.sel(time=time_sel)

            #Read calendar
            with xr.open_dataset(fnames[0], decode_times=False) as ds:
                calendar = ds.time.attrs['calendar']
                ds.close()            

            #Drop unnecessary variables
            vars_drop = set(data.data_vars).difference([variab])
            for var in vars_drop:  data = data.drop(var)
            
            #Convert °C to K (if necessary)
            if (model[0]=='CNRM-CERFACS-CNRM-CM5') and (model[1]=='CNRM-ALADIN53') and (variab in ['tasmax', 'tasmin']):
                
                sel_time = (data.time.dt.year>=2006) * 273.15
                attrs = data[variab].attrs
                data[variab] = data[variab] + sel_time
                data[variab].attrs = attrs
                
            #Check units of huss
            check1 = data[variab].isel(time=slice(0, 100)).mean()
            check2 = data[variab].isel(time=slice(-100, -1)).mean()
            if (variab=='huss') and ((check1>0.01) or (check1<0.0001) or (check2>0.01) or (check2<0.0001)):
                sys.exit('Check units of ' + variab)
                
            #Select summer
            sel_JJA  = (data.time.dt.month>=6) & (data.time.dt.month<=8)
            data_JJA = data.isel(time=sel_JJA)

            #Get climatology for summer months
            month_sta = 6
            month_end = 8
            win_half  = 15
            climatology = fun_HWC.get_clim_or_quant('climatology', data, calendar, [variab], month_sta, month_end, win_half)

            #Select original data in JJA and adjust time (for matching DOY)
            data_JJA = fun_HWC.replace_time(data_JJA, calendar, month_sta, month_end)

            #Calculate JJA anomalies
            JJA_anom = data_JJA.groupby("time.dayofyear") - climatology.groupby("time.dayofyear").mean()

            #Calculate summer mean, median, Q90, Q99
            data_JJA_mean = data_JJA.mean('time')
            data_JJA_Q50  = data_JJA.quantile(0.50, 'time')
            data_JJA_Q90  = data_JJA.quantile(0.90, 'time')
            data_JJA_Q99  = data_JJA.quantile(0.99, 'time')

            #Calculate summer variability
            JJA_anom_Q25 = JJA_anom.quantile(0.25, 'time')
            JJA_anom_Q75 = JJA_anom.quantile(0.75, 'time')
            JJA_anom_IQR = JJA_anom_Q75 - JJA_anom_Q25

            #Calculate summer average, yearly maximum, and summer variability
            data_Yearmax = data.resample(time='1Y').max()            

            #Creat output directory
            dir_JJA_out  =  dir_JJA + variab + '/'
            dir_Ymax_out =  dir_Yearmax + variab + '/'
            if not os.path.exists(dir_JJA_out):   os.mkdir(dir_JJA_out)
            if not os.path.exists(dir_Ymax_out):  os.mkdir(dir_Ymax_out)

            #Create output file names
            output_str = "_" + "_".join(model) + '_' + "-".join(scenarios) +  "_" + time_sel.start + "-" + time_sel.stop + ".nc"
            fname_JJA_mean = dir_JJA_out + variab + "_JJA-mean" + output_str
            fname_JJA_Q50  = dir_JJA_out + variab + "_JJA-Q50" + output_str
            fname_JJA_Q90  = dir_JJA_out + variab + "_JJA-Q90" + output_str
            fname_JJA_Q99  = dir_JJA_out + variab + "_JJA-Q99" + output_str
            fname_JJQ_IQR  = dir_JJA_out + variab + "-anom_JJA-IQR" + output_str
            fname_Yearmax  = dir_Ymax_out + variab + "_Yearmax" +  output_str
            if os.path.exists(fname_JJA_mean):  os.remove(fname_JJA_mean)
            if os.path.exists(fname_JJA_Q50):   os.remove(fname_JJA_Q50)
            if os.path.exists(fname_JJA_Q90):   os.remove(fname_JJA_Q90)
            if os.path.exists(fname_JJA_Q99):   os.remove(fname_JJA_Q99)
            if os.path.exists(fname_JJQ_IQR):   os.remove(fname_JJQ_IQR)
            if os.path.exists(fname_Yearmax):   os.remove(fname_Yearmax)

            #Save in file
            data_JJA_mean.to_netcdf(fname_JJA_mean)
            data_JJA_Q50.to_netcdf(fname_JJA_Q50)
            data_JJA_Q90.to_netcdf(fname_JJA_Q90)
            data_JJA_Q99.to_netcdf(fname_JJA_Q99)
            JJA_anom_IQR.to_netcdf(fname_JJQ_IQR)
            data_Yearmax.to_netcdf(fname_Yearmax)

            #Remove temporarily merged file
            os.remove(file_merge)
